In [1]:
!conda env list

# conda environments:
#
base                     /home/goswami.p/miniconda3
cs5330_pa5_1             /home/goswami.p/miniconda3/envs/cs5330_pa5_1
dicl                     /home/goswami.p/miniconda3/envs/dicl
ezflow                *  /home/goswami.p/miniconda3/envs/ezflow
flow                     /home/goswami.p/miniconda3/envs/flow
inpaint                  /home/goswami.p/miniconda3/envs/inpaint



In [2]:
!nvidia-smi

Tue Dec  6 18:22:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   37C    P0    55W / 300W |      0MiB / 32768MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# A Novel Optical Flow model with alternating layers of Self Attention and Cross Attention

In [1]:
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from nnflow import SCCFlow
from ezflow.models import build_model

In [2]:
def count_params(model):
    return str(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000) + "M params"

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

___

In [11]:
model_0 = build_model('SCCFlow', cfg_path='../configs/sccflow/models/sccflow_v00.yaml', custom_cfg=True)

In [12]:
count_params(model_0)

'1.364998M params'

In [26]:
model_0 = model_0.to(device)

In [27]:
img1 = torch.randn(1,3,368,496)
img2 = torch.randn(1,3,368,496)

img1 = img1.to(device)
img2 = img2.to(device)
img1.device

device(type='cuda', index=0)

In [28]:
results = model_0(img1, img2)
results['flow_preds'][0].shape, results['flow_preds'][1].shape

(torch.Size([1, 2, 368, 496]), torch.Size([1, 2, 368, 496]))

In [15]:
model_cfg = get_cfg("../configs/transformers/models/crossflow_v02.yaml")
training_cfg = get_cfg("../configs/transformers/trainer/kubrics_v3_5.yaml")

training_cfg.DATA.BATCH_SIZE = 10
training_cfg.NUM_STEPS = 50000
training_cfg.SCHEDULER.PARAMS.total_steps = 50100
training_cfg.LOG_DIR = "../../results/crossflow/logs/exp005"
training_cfg.CKPT_DIR = "../../results/crossflow/ckpts/exp005"
training_cfg.FREEZE_BATCH_NORM = False

pp.pprint(training_cfg)

NameError: name 'get_cfg' is not defined

In [14]:
train_loader_creator = CustomDataloaderCreator(
    batch_size=training_cfg.DATA.BATCH_SIZE,
    num_workers=training_cfg.DATA.NUM_WORKERS,
    pin_memory=training_cfg.DATA.PIN_MEMORY,
    append_valid_mask=training_cfg.DATA.APPEND_VALID_MASK,
    shuffle=training_cfg.DATA.SHUFFLE
)

val_loader_creator = CustomDataloaderCreator(
    batch_size=training_cfg.DATA.BATCH_SIZE,
    num_workers=training_cfg.DATA.NUM_WORKERS,
    pin_memory=training_cfg.DATA.PIN_MEMORY,
    append_valid_mask=training_cfg.DATA.APPEND_VALID_MASK,
    shuffle=training_cfg.DATA.SHUFFLE
)

In [15]:
train_loader_creator.add_Kubric(
    root_dir="../../KubricFlow",
    crop=True,
    crop_type="random",
    crop_size=training_cfg.DATA.TRAIN_CROP_SIZE,
    augment=training_cfg.DATA.AUGMENTATION.USE,
    aug_params={
        "spatial_aug_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.SPATIAL_AUG_PARAMS,
        "color_aug_params":training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.COLOR_AUG_PARAMS,
        "eraser_aug_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.ERASER_AUG_PARAMS,
        "noise_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.NOISE_PARAMS,
        "spatial_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.AUTOFLOW_SPATIAL_PARAMS,
        "translate_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.TRANSLATE_PARAMS,
        "rotate_params": training_cfg.DATA.AUGMENTATION.PARAMS.TRAINING.ROTATE_PARAMS
    },
    norm_params=training_cfg.DATA.NORM_PARAMS
)

In [16]:
val_loader_creator.add_MPISintel(
    root_dir="../../../../Datasets/MPI_Sintel/",
    split="training",
    crop=True,
    crop_type="center",
    crop_size=training_cfg.DATA.VAL_CROP_SIZE,
    augment=False,
    norm_params=training_cfg.DATA.NORM_PARAMS
)

In [17]:
model = CrossFlow(model_cfg)

In [18]:
count_params(model)

'1.684236M params'

In [19]:
trainer = Trainer(
    training_cfg, 
    model, 
    train_loader = train_loader_creator.get_dataloader(), 
    val_loader = val_loader_creator.get_dataloader()
)

Total image pairs loaded: 131950/131951

Total image pairs loaded: 1040/1041



In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp001
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/transformers/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 49.07904767990112
Iterations: 200, Total iterations: 200, Average batch training loss: 34.91838252544403
Iterations: 300, Total iterations: 300, Average batch training loss: 30.086434790293374
Iterations: 400, Total iterations: 400, Average batch training loss: 27.542579951286317
Iterations: 500, Total iterations: 500, Average batch training loss: 25.68077686882019
Iterations: 600, Total iterations: 600, Average batch training loss: 24.51905212243398
Iterations: 700, Total iterations: 700, Average batch training loss: 23.689530146462577
Iterations: 800, Total iterations: 800, Average batch training loss: 23.024980469942093
Iterations: 900, Total iterations: 900, Average batch training loss: 22.591071795887416
Iterations: 1000, Total iterations: 1000, Average batch training loss: 22.220978223323822

 --------------------------------------------------------------------------------

Iteration 1000: Average validation los

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp002
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/transformers/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 47.402778425216674
Iterations: 200, Total iterations: 200, Average batch training loss: 34.730524196624756
Iterations: 300, Total iterations: 300, Average batch training loss: 30.00316294034322
Iterations: 400, Total iterations: 400, Average batch training loss: 27.572381217479705
Iterations: 500, Total iterations: 500, Average batch training loss: 25.961023359298707
Iterations: 600, Total iterations: 600, Average batch training loss: 24.760241158803304
Iterations: 700, Total iterations: 700, Average batch training loss: 23.832342450278144
Iterations: 800, Total iterations: 800, Average batch training loss: 23.23447922706604
Iterations: 900, Total iterations: 900, Average batch training loss: 22.79281840854221
Iterations: 1000, Total iterations: 1000, Average batch training loss: 22.250091296195983

 --------------------------------------------------------------------------------

Iteration 1000: Average validation lo

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp003
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 39.230209560394286
Iterations: 200, Total iterations: 200, Average batch training loss: 30.188817505836486
Iterations: 300, Total iterations: 300, Average batch training loss: 27.124033584594727
Iterations: 400, Total iterations: 400, Average batch training loss: 25.11518214941025
Iterations: 500, Total iterations: 500, Average batch training loss: 23.938732753753662
Iterations: 600, Total iterations: 600, Average batch training loss: 22.988590288162232
Iterations: 700, Total iterations: 700, Average batch training loss: 22.328436083112443
Iterations: 800, Total iterations: 800, Average batch training loss: 21.879796508550644
Iterations: 900, Total iterations: 900, Average batch training loss: 21.528786935806274
Iterations: 1000, Total iterations: 1000, Average batch training loss: 21.203034631729125

 --------------------------------------------------------------------------------

Iteration 1000: Average validation 

In [ ]:
trainer.train()

Loss function: SequenceLoss is initialized!
Optimizer: AdamW is initialized!
Scheduler: OneCycleLR is initialized!
Training config:

CKPT_DIR: ../../results/crossflow/ckpts/exp004
CKPT_INTERVAL: 20000
CRITERION:
  CUSTOM: True
  NAME: SequenceLoss
  PARAMS:
    gamma: 0.8
    max_flow: 400.0
DATA:
  APPEND_VALID_MASK: True
  AUGMENTATION:
    PARAMS:
      TRAINING:
        AUTOFLOW_SPATIAL_PARAMS:
          black: False
          enabled: False
          rot: [0.4, 0.03]
          scale: [0.3, 0.03, 0.1]
          squeeze: [0.3, 0.0]
          trans: [0.4, 0.03]
        COLOR_AUG_PARAMS:
          aug_prob: 0.2
          brightness: 0.4
          contrast: 0.4
          enabled: True
          hue: 0.15915494309189535
          saturation: 0.4
        ERASER_AUG_PARAMS:
          aug_prob: 0.5
          bounds: [50, 100]
          enabled: True
        NOISE_PARAMS:
          aug_prob: 0.5
          enabled: False
          noise_std_range: 0.06
        ROTATE_PARAMS:
          aug_pr

/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/nnflow-0.0.0-py3.7.egg/nnflow/models/transformers/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/home/goswami.p/miniconda3/envs/ezflow/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811797118/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Iterations: 100, Total iterations: 100, Average batch training loss: 38.806310939788816
Iterations: 200, Total iterations: 200, Average batch training loss: 29.86527271270752
Iterations: 300, Total iterations: 300, Average batch training loss: 26.4670383644104
Iterations: 400, Total iterations: 400, Average batch training loss: 24.626682333946228
Iterations: 500, Total iterations: 500, Average batch training loss: 23.34748571586609
Iterations: 600, Total iterations: 600, Average batch training loss: 22.67428199450175
Iterations: 700, Total iterations: 700, Average batch training loss: 22.064218209130424
Iterations: 800, Total iterations: 800, Average batch training loss: 21.582395669817924
Iterations: 900, Total iterations: 900, Average batch training loss: 21.205847839249504
Iterations: 1000, Total iterations: 1000, Average batch training loss: 20.946390319347383

 --------------------------------------------------------------------------------

Iteration 1000: Average validation loss

In [16]:
model_1 = build_model('SCCFlow', cfg_path='../configs/sccflow/models/sccflow_v01.yaml', custom_cfg=True)

In [17]:
count_params(model_1)

'1.684236M params'

In [19]:
# model_1

In [20]:
img1 = torch.randn(1,3,368,496)
img2 = torch.randn(1,3,368,496)

results = model_1(img1, img2)
results['flow_preds'][0].shape, results['flow_preds'][1].shape

(torch.Size([1, 2, 368, 496]), torch.Size([1, 2, 368, 496]))

In [21]:
model_2 = build_model('SCCFlow', cfg_path='../configs/sccflow/models/sccflow_v02.yaml', custom_cfg=True)

In [23]:
# model_2

In [24]:
count_params(model_2)

'1.684236M params'

In [25]:
img1 = torch.randn(1,3,368,496)
img2 = torch.randn(1,3,368,496)

results = model_2(img1, img2)
results['flow_preds'][0].shape, results['flow_preds'][1].shape

(torch.Size([1, 2, 368, 496]), torch.Size([1, 2, 368, 496]))